In [1]:
import scipy
from pathlib import Path
import pathlib
from scipy.io.wavfile import read
from scipy.signal import hann
from scipy import signal
from scipy.fftpack import rfft
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
from yaafelib import *
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")

#cols=['track_name','bpm','harmonic_rythmic_ratio',\
#'spectral_centroid','spectral_flatness','count_delta_above_mean',\
#'ratio_above_rmse_mean','max_rmse','mean_rms','mean_zcr','liked']
df = pd.DataFrame()

In [2]:
def set_output(artist):
 
    path = pathlib.Path("~/Music/" + artist + "/*.mp3").expanduser()
    parts = path.parts[1:] if path.is_absolute() else path.parts
    return pathlib.Path(path.root).glob(str(pathlib.Path("").joinpath(*parts)))

def other_features(X,sample_rate): # 
    
    cqt = np.mean(librosa.amplitude_to_db(librosa.cqt(y, sr=sr), ref=np.max).T,axis=0)
    return cqt
 


In [3]:
%%time
# This takes way to long, consider compiling in C or parallel processing
# or some other method

artist = 'Metallica'
output = set_output(artist)
tracks = []
for p in output: 
    track_name = str(p).replace('/Users/mcassettix/Music/' + artist + '/','')
    print(track_name)
    try:
        y_total, sr = librosa.load(p) 
    except:
        print('loading issue, skipping track',p)
        continue
    
    #Sample each track multiple times to increase samples
    #Take 30 second slices of a song, rounded down 
    times = librosa.frames_to_time(np.arange(len(y_total)))
    if  times[-1] > 30: #standarizing length to 2 minutes
        max_time = len(y_total)/sr
        num_slices = math.floor(max_time/30) 
        hop_length = num_slices * sr
        max_x = num_slices*30*sr        
    else:
        print('Warning: you need pick a time range')
        print('skipping track ',p)
        continue 
    frames = list(range(hop_length,max_x, hop_length))

    
    for index in range(len(frames)-1):
        tracks.append(track_name)
        y = y_total[frames[index]:frames[index + 1]]
        #tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        #print('Estimated tempo: {:.2f} beats per minute'.format(tempo))

        df = pd.DataFrame(np.transpose(other_features(y,sr)).reshape(1,-1)).append(df, ignore_index=True)

df['track_names'] = tracks
df['band'] = artist


06 - Live At Wembley Stadium London 1.mp3
02 - Wherever I May Roam.mp3
03 - Some Kind Of Monster.mp3
09 - Last Caress.mp3
05 - One.mp3
04 - Dirty Window.mp3
09 - Poor Twisted Me.mp3
11 - Low Man's Lyric.mp3
10 - Devil's Dance.mp3
05 - All Nightmare Long.mp3
15 - For Whom The Bell Tolls (Live).mp3
08 - The Judas Kiss.mp3
07 - Killing Time.mp3
10 - Purify.mp3
07 - Tuesday's Gone.mp3
10 - Prince Charming.mp3
05 - A) Last Caress; B) Green Hell.mp3
08 - Justice Medley.mp3
08 - One.mp3
10 - The Wait.mp3
12 - Attitude.mp3
05 - (Anesthesia) Pulling Teeth.mp3
03 - Sabbra Cadabra.mp3
08 - Nothing Else Matters.mp3
06 - Seek And Destroy.mp3
05 - Seek & Destroy.mp3
02 - The Call Of Ktulu.mp3
03 - Eye Of The Beholder.mp3
18 - Whiplash.mp3
10 - Last Caress.mp3
16 - Stone Dead Forever.mp3
06 - Harvester Of Sorrow.mp3
09 - Where The Wild Things Are.mp3
09 - Suicide & Redemption.mp3
17 - A) Prowler B) Run To The Hills.mp3
04 - Turn The Page.mp3
03 - For Whom The Bell Tolls.mp3
05 - Last Caress - Green H

In [4]:
df.to_pickle('metallica_model2.pkl')